## Model 1.0
Powered by [Eleonora Priori](https://www.est-en.unito.it/do/docenti.pl/Alias?eleonora.priori#tab-profilo) and [Pietro Terna](https://terna.to.it/) 


In [1]:
%%javascript
// to avoid scroll in windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

====================================================================================================

## 1

import libs \
MPI init \
context and runner definition \
t(), T(), Tc() function definitions \
random number generator rng creation \
initialization of the parameters from yaml file\
memory allocations to manage ghosts


====================================================================================================

In [2]:
import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters
from repast4py import schedule
from repast4py import core
from typing import Tuple, List, Dict
import numpy as np
import csv


comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() 

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# Initialize the default schedule runner, HERE to create the t() function,
# returning the tick value
runner = schedule.init_schedule_runner(comm)

# tick number
def t():
    return int(runner.schedule.tick)

#Initializes the repast4py.parameters.params dictionary with the model input parameters.
params = parameters.init_params("model1.yaml", "")


#generate random seed
repast4py.random.init(rng_seed=params['myRandom.seed'][rank]) #each rank has a seed
rng = repast4py.random.default_rng 


#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime
T()

#cpuTimer Tc()
startCpuTime=-1
def Tc():
    global startCpuTime
    if startCpuTime < 0:
        startCpuTime=time.process_time()
    return time.process_time() - startCpuTime
Tc()

agent_cache={} # dict with uid as keys and agents' tuples as values

===================================================================================================

## 2

create agents' classes and restore_agent function 



===================================================================================================

In [3]:
class Firm(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, labor:int, capital:float,\
                 duration:int, recipe: float, laborProductivity: float, maxOrderProduction: float,\
                 dimensionalClass: int):
        super().__init__(id=local_id, type=Firm.TYPE, rank=rank) #uid
        self.labor=labor
        self.capital=capital
        self.duration=duration
        self.recipe = recipe
        self.laborProductivity=laborProductivity
        self.maxOrderProduction=maxOrderProduction
        self.dimensionalClass=dimensionalClass
        self.lostProduction=0
        self.inventories=0
        self.inProgressInventories=0
        self.appRepository=[] #aPP=aProductiveProcess
        self.availableLabor=self.labor
        self.availableCapital=self.capital
        
        self.profits=0
        self.revenues=0
        self.totalCosts=0
        self.addedValue=0
        self.finalInventories=0
        self.initialInventories=0
        
        self.myBalancesheet=np.zeros((params['howManyCycles'], 6))

        
    def produce(self, productionOrder:float)->tuple:
        
        self.costOfProductionOrder=0
        output=0
        self.costOfUnusedFactors=0
        totalLostProduction=0
        totalCostOfLostProduction=0
        
        
        lastOrderProductionInEachPeriod=productionOrder/self.duration
        requiredLabor=np.ceil(lastOrderProductionInEachPeriod/self.laborProductivity)
        requiredCapital=requiredLabor*self.recipe
        
        #TMP TMP TMP
        #print(self.uid,"last in",productionOrder,productionOrder/self.duration,\
               #requiredLabor,requiredCapital)

        #create a new aPP or skip the order
        if requiredLabor <= self.labor and requiredCapital <= self.capital: 
            aProductiveProcess = ProductiveProcess(lastOrderProductionInEachPeriod, \
                                                   requiredLabor, requiredCapital, self.duration)
            self.appRepository.append(aProductiveProcess)
        
        for aProductiveProcess in self.appRepository:  
            
            if aProductiveProcess.hasResources or \
            (self.availableLabor >= aProductiveProcess.requiredLabor and\
                self.availableCapital >= aProductiveProcess.requiredCapital):
                
                if not aProductiveProcess.hasResources:
                    self.availableLabor -= aProductiveProcess.requiredLabor
                    self.availableCapital -= aProductiveProcess.requiredCapital
                    aProductiveProcess.hasResources = True 
                #production
                (outputOfThePeriod, requiredLabor, requiredCapital,\
                lostProduction, costOfLostProduction)=aProductiveProcess.step(productionOrder)               
                     
                output += outputOfThePeriod
                
                cost = requiredLabor*params['wage'] + requiredCapital*params['costOfCapital']
                self.costOfProductionOrder += cost
                
                totalLostProduction += lostProduction
                totalCostOfLostProduction += costOfLostProduction
              
        
                if aProductiveProcess.failure:
                    self.inProgressInventories -= cost*(aProductiveProcess.productionClock-1)
                
                else:
                    if aProductiveProcess.productionClock < aProductiveProcess.duration:
                        self.inProgressInventories += cost
                    else:
                        self.inventories+=cost*self.duration
                        self.inProgressInventories -= cost*(self.duration-1)

        self.costOfUnusedFactors =  self.availableLabor*params['wage'] + \
                                    self.availableCapital*params['costOfCapital']
                             
        
        # remove concluded aPPs from the list
        for i in range(len(self.appRepository)-1,-1,-1):
            if self.appRepository[i].productionClock == self.appRepository[i].duration: 
                self.availableLabor+=self.appRepository[i].requiredLabor
                self.availableCapital+=self.appRepository[i].requiredCapital
                #self.appRepository.remove(aProductiveProcess)
                del self.appRepository[i]

        
        #print(self.uid,"\n", productionOrder,  self.labor, self.capital, self.recipe,\
              #self.laborProductivity,self.duration,"\n",\
              #output, costOfProductionOrder, self.costOfUnusedFactors, self.inventories,\
              #self.inProgressInventories, totalLostProduction,totalCostOfLostProduction, flush=True)
        
        return(output, self.costOfProductionOrder, self.costOfUnusedFactors,self.inventories,\
               self.inProgressInventories, totalLostProduction, totalCostOfLostProduction)
    
    def makeBalancesheet(self):
        if t()==0: self.initialInventories=0 
        else: self.initialInventories=self.finalInventories
        
        self.totalCosts=self.costOfProductionOrder + self.costOfUnusedFactors
        self.finalInventories=self.inventories+self.inProgressInventories
        self.profits=self.finalInventories-self.totalCosts-self.initialInventories
        self.addedValue=self.profits+self.totalCosts
        
        self.myBalancesheet[t(), 0]=self.dimensionalClass
        
        self.myBalancesheet[t(), 1]=self.initialInventories
        self.myBalancesheet[t(), 2]=self.totalCosts
        self.myBalancesheet[t(), 3]=self.finalInventories
        self.myBalancesheet[t(), 4]=self.profits
        
        self.myBalancesheet[t(), 5]=self.addedValue
        
    
###########################################################################################################################


class ProductiveProcess():
    def __init__(self, targetProductionOfThePeriod:float, requiredLabor:int, requiredCapital:float, duration:int):
        
        self.targetProductionOfThePeriod=targetProductionOfThePeriod
        self.requiredLabor = requiredLabor
        self.requiredCapital = requiredCapital
        self.duration = duration
        self.productionClock=0
        self.hasResources= False
        
    def step(self, productionOrder)->tuple:
        
        lostProduction=0
        self.costOfLostProduction=0
        self.productionClock += 1
        self.failure=False
        
        # production failure
        if params['probabilityToFailProductionChoices'] >= rng.random():
            self.failure=True
            #print("failure",flush=True)
            lostProduction=self.targetProductionOfThePeriod*self.productionClock
            self.targetProductionOfThePeriod=0
            self.costOfLostProduction=(params['wage']* self.requiredLabor+\
                                       params['costOfCapital']* self.requiredCapital)*self.productionClock
            self.duration = self.productionClock   

        return(self.targetProductionOfThePeriod, self.requiredLabor, self.requiredCapital, \
               lostProduction, self.costOfLostProduction)

===================================================================================================

## 3

the model

===================================================================================================

In [4]:
class Model:
    
    global params
    PARAMS = params
    
    def __init__(self, params: Dict):
        
        self.totalProduction=[]
        self.totalCostOfProduction=[]
        self.totalCostOfUnusedFactors=[]
        self.totalInventories=[]
        self.totalInProgressInventories=[]
        self.totalLostProduction=[]
        self.totalCostOfLostProduction=[]
        
        #the context and the runner are created in step 1 
      
        runner.schedule_event(          0.0,     self.initGhosts) 
        runner.schedule_repeating_event(0.0,  1, self.counter)
        runner.schedule_repeating_event(0.1,  1, self.firmsProducing)
        runner.schedule_repeating_event(0.2,  1, self.enterprisesMakingBalancesheet) #enterprises=firms+banks
        
        runner.schedule_stop(params['howManyCycles'])
        runner.schedule_end_event(self.finish)
        
        ####################################################################################################
        ###################################### CREATE FIRM AGENTS ##########################################
        ####################################################################################################
        
        #importing csv file containing info about firms 
        #share of firms of that class, L-min, L-max, K-min, K-max, t-min, t-max, recipe, L-prod
        with open('firm-features.csv', newline='') as csvfile:
            firmReader= csv.reader(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
            
            self.rowNumber=0
            k=0
            #for each record in firm-features.csv
            for row in firmReader:
                print(row)
                for i in range(int(row[0] * params['Firm.count'])// rankNum):
                    labor= rng.integers(row[1], row[2]+1)
                    capital= row[3] + rng.random()*(row[4] -row[3])
                    duration= rng.integers(row[5], row[6]+1)
                    recipe= row[7] #K/L 
                    laborProductivity= row[8]
                    maxOrderProduction= row[9]
                    dimensionalClass=self.rowNumber
                    aFirm =Firm(k, rank, labor, capital, duration, recipe, laborProductivity,\
                                maxOrderProduction, dimensionalClass)
                    context.add(aFirm)
                    k += 1
                self.rowNumber += 1

        
    #initialize ghosts by sending them in the ranks before starting the simulation
    def initGhosts(self):
        pass
    
    #count the cycles number
    def counter(self):
        if int(t()) % params["tickNumber.betweenChecks"] == 0: 
            print("rank", rank, "tick", t(), flush=True)
            
  
    
    def firmsProducing(self):
        self.totalProduction.append([0]*(self.rowNumber)) #for each cycle adds a sub-list of lenght number of dimClass
        self.totalCostOfProduction.append([0]*(self.rowNumber))
        self.totalCostOfUnusedFactors.append([0]*(self.rowNumber))
        self.totalInventories.append([0]*(self.rowNumber))
        self.totalInProgressInventories.append([0]*(self.rowNumber))
        self.totalLostProduction.append([0]*(self.rowNumber))
        self.totalCostOfLostProduction.append([0]*(self.rowNumber))
        
        for aFirm in context.agents(agent_type=0):
            #print(aFirm.dimensionalClass)
            tupleOfProductionResults = aFirm.produce(rng.random()*aFirm.maxOrderProduction)

            self.totalProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[0]
            self.totalCostOfProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[1]
            self.totalCostOfUnusedFactors[t()][aFirm.dimensionalClass] += tupleOfProductionResults[2]
            self.totalInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[3]
            self.totalInProgressInventories[t()][aFirm.dimensionalClass] += tupleOfProductionResults[4] 
            self.totalLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[5]
            self.totalCostOfLostProduction[t()][aFirm.dimensionalClass] += tupleOfProductionResults[6]
            
    def enterprisesMakingBalancesheet(self):
        for aFirm in context.agents(agent_type=0):
            aFirm.makeBalancesheet()
    
    
    #finish
    def finish(self):
        
        # infos for data_analysis.ipynb
        with open('plotInfo.csv', 'w', newline='')\
          as file:
            writer = csv.writer(file)
            writer.writerow((params["log_file_root"],rankNum,\
                             context.size(agent_type_ids=[0])[0]))
        
        """
        print("\n total production", self.totalProduction, flush=True)
        print("\n total cost of production", self.totalCostOfProduction, flush=True)
        print("\n total cost of unused factors", self.totalCostOfUnusedFactors, flush=True)
        print("\n total inventories", self.totalInventories, flush=True)
        print("\n total in progress inventories", self.totalInProgressInventories, flush=True)
        print("\n total lost production", self.totalLostProduction, flush=True)
        print("\n total cost of lost production", self.totalCostOfLostProduction, flush=True)     
        """
        
        names=["_total_production_","_total_cost_of_production_","_total_cost_of_unused_factors_",
               "_total_inventories_","_total_in_progress_inventories_",
               "_total_lost_production_","_total_cost_of_lost_production_"]
        contents=[self.totalProduction,self.totalCostOfProduction,
                  self.totalCostOfUnusedFactors,
                  self.totalInventories,self.totalInProgressInventories,
                  self.totalLostProduction,self.totalCostOfLostProduction]
        
        for s in range(len(names)):
            with open(params["log_file_root"]+names[s]+str(rank)+'.csv', 'w', newline='') as file:
                writer = csv.writer(file)
                for k in range(params["howManyCycles"]):
                    writer.writerow(contents[s][k])
                    
        fileList=[] 
        
        for aFirm in context.agents(agent_type=0):        
            #save balancesheets
            with open(params["log_file_root"]+'_'+str(aFirm.uid)+'.csv', 'w', newline='') as file:
                np.savetxt(file, aFirm.myBalancesheet, delimiter=",")
                #create the list of the files containing the balancesheets
                fileList.append(params["log_file_root"]+'_'+str(aFirm.uid)+'.csv')
        
        #save list of balancesheets file names
        with open(params["log_file_root"]+'_balancesheetList_'+str(rank)+'.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(fileList)
         
        print("THE END!", flush=True)
    
    def start(self):
        runner.execute()

=========================================================================================================

## 4

run the model

==========================================================================================================

In [5]:
def run(params: Dict):
    
    model = Model(params) 
    model.start()
    
run(params)

[0.1, 40.0, 80.0, 200.0, 400.0, 5.0, 10.0, 5.0, 0.8, 100.0]
[0.3, 20.0, 50.0, 60.0, 200.0, 2.0, 4.0, 2.0, 0.7, 50.0]
[0.6, 2.0, 18.0, 1.0, 8.0, 1.0, 1.0, 0.5, 0.6, 10.0]
rank 0 tick 0
rank 0 tick 1
rank 0 tick 2
rank 0 tick 3
rank 0 tick 4
rank 0 tick 5
rank 0 tick 6
rank 0 tick 7
rank 0 tick 8
rank 0 tick 9
rank 0 tick 10
THE END!
